In [ ]:
import pandas as pd
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(palette='summer')

In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 9.1 MB/s eta 0:00:00


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.7 MB/s eta 0:00:00


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.6 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.0 MB/s eta 0:00:00


In [ ]:
import transformers
from datasets import load_dataset
import evaluate

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Подготовка данных

In [ ]:
billsum = load_dataset("billsum", split="ca_test")

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

Dataset billsum downloaded and prepared to /root/.cache/huggingface/datasets/billsum/default/3.0.0/75cf1719d38d6553aa0e0714c393c74579b083ae6e164b2543684e3e92e0c4cc. Subsequent calls will reuse this data.


In [ ]:
billsum

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 1237
})

In [ ]:
billsum = billsum.train_test_split(test_size=0.1)

In [ ]:
billsum

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1113
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 124
    })
})

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained("ainize/bart-base-cnn")

In [ ]:
def preprocess_function(examples):
    model_inputs = tokenizer(examples["text"], max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/1113 [00:00<?, ? examples/s]

Map:   0%|          | 0/124 [00:00<?, ? examples/s]

In [ ]:
model = transformers.AutoModelForSeq2SeqLM.from_pretrained("ainize/bart-base-cnn")

In [ ]:
data_collator = transformers.DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
training_args = transformers.Seq2SeqTrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=2,
    )

In [ ]:
trainer = transformers.Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,2.054266
2,2.174500,2.005796


TrainOutput(global_step=558, training_loss=2.1474227221635935, metrics={'train_runtime': 403.3054, 'train_samples_per_second': 5.519, 'train_steps_per_second': 1.384, 'total_flos': 1357273356042240.0, 'train_loss': 2.1474227221635935, 'epoch': 2.0})

###  Предсказания на тестовом множестве

In [ ]:
text_example = billsum["test"]['text'][0]
print(text_example)

The people of the State of California do enact as follows:


SECTION 1.
Section 236.14 is added to the Penal Code, to read:
236.14.
(a) If a person was arrested for or convicted of any nonviolent offense committed while he or she was a victim of human trafficking, including, but not limited to, prostitution as described in subdivision (b) of Section 647, the person may petition the court for vacatur relief of his or her convictions and arrests under th">(e) The court may, with the agreement of the petitioner and all of the involved state or local prosecutorial agencies, consolidate into one hearing a petition with multiple convictions from different jurisdictions.
(f) If the petition is opposed or if the court otherwise deems it necessary, the court shall schedule a hearing on the petition. The hearing may consist of the following:
(1) Testimony by the petitioner, which may be required in support of the petition.
(2) Evidence and supporting documentation in support of the petition.
(3)

In [ ]:
input_ids = tokenizer.encode(
    text_example,
    return_tensors="pt",
    max_length=1024,
    truncation=True,
    ).to(device)

In [ ]:
input_ids.shape

torch.Size([1, 1024])

In [ ]:
summary_text_ids = model.generate(
    input_ids=input_ids,
    bos_token_id=model.config.bos_token_id,
    eos_token_id=model.config.eos_token_id,
    max_length=142,
    min_length=56,
    num_beams=4,
)

In [ ]:
summary_text_ids

tensor([[    2,     0,  9089, 15528,   488,  2730,  7396,    10,   621,    54,
            21,  1128,    13,    50,  3828,     9,   143, 36887,  2970,  2021,
           150,    37,    50,    79,    21,    10,  1802,     9,  1050,  7492,
             6,   217,     6,    53,    45,  1804,     7,     6, 23105,     6,
             7,  5265,     5,   461,    13, 18721,  8367,  3500,     9,    39,
            50,    69, 12618,     8,  7102,     4,  3015, 15528,   488,  3441,
             5,   461,     7, 19752,    88,    65,  1576,    10,  5265,    19,
          1533, 12618,    31,   430, 17607,     4, 50118,   713,  1087,    74,
          2703,     5,   461,     6,    19,     5,  1288,     9,     5, 31390,
             8,    70,     9,     5,   963,   194,    50,   400, 42308, 17707,
          2244,     6,     7,     2]], device='cuda:0')

In [ ]:
decoded_text = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

In [ ]:
len(decoded_text), len(text_example)

(547, 8857)

In [ ]:
decoded_text

'Existing law authorizes a person who was arrested for or convicted of any nonviolent offense committed while he or she was a victim of human trafficking, including, but not limited to, prostitution, to petition the court for vacatur relief of his or her convictions and arrests. Existing law requires the court to consolidate into one hearing a petition with multiple convictions from different jurisdictions.\nThis bill would require the court, with the agreement of the petitioner and all of the involved state or local prosecutorial agencies, to'

In [ ]:
summaries = []

for text in tqdm(billsum["test"]['text']):
    input_ids = tokenizer.encode(
        text,
        return_tensors="pt",
        max_length=1024,
        truncation=True,
        ).to(device)

    summary_text_ids = model.generate(
        input_ids=input_ids,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        max_length=142,
        min_length=56,
        num_beams=4,
    )

    decoded_text = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)
    summaries.append(decoded_text)

  0%|          | 0/124 [00:00<?, ?it/s]

## Считаем качество

### ROUGE

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=71e95dac241dd35a66fc8195d2717a975f9a1112f7431ca946d0133aafada3cd
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
rouge = evaluate.load('rouge')

In [ ]:
%%time
results = rouge.compute(
        predictions=summaries,
        references=billsum["test"]['summary']
    )

CPU times: user 5.03 s, sys: 8.51 ms, total: 5.03 s
Wall time: 5.12 s


In [ ]:
results

{'rouge1': 0.3295094036218852,
 'rouge2': 0.1785330127243029,
 'rougeL': 0.22409067060892754,
 'rougeLsum': 0.28382496411321023}